In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv("data\\diabetes.csv")
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [5]:
### No Missing Values
df.duplicated().sum()  
### No duplicates

0

In [6]:
df.corr()['Outcome']   ### Correlation with input features

Pregnancies                 0.221898
Glucose                     0.466581
BloodPressure               0.065068
SkinThickness               0.074752
Insulin                     0.130548
BMI                         0.292695
DiabetesPedigreeFunction    0.173844
Age                         0.238356
Outcome                     1.000000
Name: Outcome, dtype: float64

In [7]:
from sklearn.preprocessing import StandardScaler

X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values
scaler = StandardScaler()


In [8]:
X = scaler.fit_transform(X)

In [9]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.2, random_state= 42)

In [53]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense, Dropout

In [20]:
model = Sequential()

model.add(Dense(32, activation= 'relu', input_dim= 8))
model.add(Dense(1, activation= 'sigmoid'))

model.compile(optimizer= 'rmsprop', loss= 'binary_crossentropy', metrics= ['accuracy'])

c:\Users\disha\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [22]:
model.fit(X_train, y_train, batch_size= 32, epochs= 10, validation_data= (X_test, y_test))

Epoch 1/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8063 - loss: 0.3898 - val_accuracy: 0.7662 - val_loss: 0.5411
Epoch 2/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8131 - loss: 0.4038 - val_accuracy: 0.7727 - val_loss: 0.5412
Epoch 3/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8174 - loss: 0.3939 - val_accuracy: 0.7727 - val_loss: 0.5418
Epoch 4/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7934 - loss: 0.4000 - val_accuracy: 0.7662 - val_loss: 0.5436
Epoch 5/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8140 - loss: 0.4185 - val_accuracy: 0.7662 - val_loss: 0.5419
Epoch 6/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8128 - loss: 0.3861 - val_accuracy: 0.7597 - val_loss: 0.5451
Epoch 7/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8261 - loss: 0.3629 - val_accuracy: 0.7597 - val_loss: 0.5453
Epoch 8/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8016 - loss: 0.4204 - val_accuracy: 0.7662 - val_loss:

### Keras tuner

In [19]:
import kerastuner as kt

C:\Users\disha\AppData\Local\Temp\ipykernel_9688\1654478174.py:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  import kerastuner as kt


In [24]:
def build_model(hp):
    model = Sequential()
    units = hp.Int('units',8, 128, step= 8)
    model.add(Dense(units= units, activation= 'relu', input_dim= 8))
    model.add(Dense(1, activation= 'sigmoid'))
    
    model.compile(optimizer= 'rmsprop', loss= 'binary_crossentropy', metrics= ['accuracy'])
    return model


In [27]:
tuner = kt.RandomSearch(build_model, 
                        objective= 'val_accuracy', 
                        max_trials= 5,
                        directory= 'mydir')

c:\Users\disha\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [28]:
tuner.search(X_train, y_train, epochs= 5, validation_data= (X_test, y_test))

Trial 5 Complete [00h 00m 02s]
val_accuracy: 0.7272727489471436

Best val_accuracy So Far: 0.7792207598686218
Total elapsed time: 00h 00m 10s


In [30]:
tuner.get_best_hyperparameters()[0].values

{'units': 88}

In [31]:
model = tuner.get_best_models(num_models= 1)[0]

c:\Users\disha\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\disha\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\saving\saving_lib.py:713: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 6 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [32]:
model.fit(X_train, y_train, batch_size= 32, epochs= 100, initial_epoch= 6)

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7729 - loss: 0.4797   
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7867 - loss: 0.4510 
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7942 - loss: 0.4464 
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 937us/step - accuracy: 0.7792 - loss: 0.4405
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7604 - loss: 0.4583 
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 987us/step - accuracy: 0.7883 - loss: 0.4409
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7927 - loss: 0.4265 
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7616 - loss: 0.4745 
Epoch 15/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8000 - loss: 0.4222 
Epoch 16/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7936 - loss: 0.4294 
Epoch 17/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7930 - loss: 0.4301 
Epoch 18/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s

In [33]:
### Number of layers

def build_model(hp):
    model = Sequential()
    model.add(Dense(88, activation= 'relu', input_dim= 8))
    
    for i in range(hp.Int('num_layers', min_value= 1, max_value= 10)):
        model.add(Dense(88, activation= 'relu'))
        
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='rmsprop', loss= 'binary_crossentropy', metrics= ['accuracy'])
    
    return model


In [34]:
tuner = kt.RandomSearch(build_model,
                        objective='val_accuracy',
                        directory= 'mydir',
                        project_name= 'num_layers')

c:\Users\disha\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [35]:
tuner.search(X_train, y_train, epochs= 5, validation_data= (X_test, y_test))

Trial 10 Complete [00h 00m 02s]
val_accuracy: 0.7857142686843872

Best val_accuracy So Far: 0.798701286315918
Total elapsed time: 00h 00m 28s


In [38]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 1}

In [39]:
model = tuner.get_best_models(num_models= 1)[0]

c:\Users\disha\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\disha\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\saving\saving_lib.py:713: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 8 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [40]:
model.fit(X_train, y_train, epochs= 100, initial_epoch= 6, validation_data= (X_test, y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.7713 - loss: 0.4905 - val_accuracy: 0.7987 - val_loss: 0.4885
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7897 - loss: 0.4527 - val_accuracy: 0.7857 - val_loss: 0.4927
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7604 - loss: 0.4493 - val_accuracy: 0.7727 - val_loss: 0.5025
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7974 - loss: 0.4150 - val_accuracy: 0.7727 - val_loss: 0.5045
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8017 - loss: 0.4171 - val_accuracy: 0.7662 - val_loss: 0.5120
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7469 - loss: 0.4630 - val_accuracy: 0.7597 - val_loss: 0.5188
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7860 - loss: 0.4582 - val_accuracy: 0.7727 - val_loss: 0.5218
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7646 - loss: 0.4256 - val_accuracy: 0.76

In [41]:
### Prone to Overfitting in this case
## Lets do multiple thingd at a time

In [60]:
def build_model(hp):
    model = Sequential()
    counter = 0
    
    for i in range(hp.Int('num_layers', 1, 10)):
        if counter == 0:
            model.add(
                Dense(
                    hp.Int('units' + str(i), 8, 128, step= 8), 
                    activation= hp.Choice('activaation' + str(i), values= ['relu', 'tanh', 'sigmoid']),
                    input_dim= 8))
            model.add(Dropout(hp.Choice('dropout' + str(i), values= [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])))
        else:
            model.add(
                Dense(
                    hp.Int('units' + str(i), 8, 128, step= 8), 
                    activation= hp.Choice('activaation' + str(i), values= ['relu', 'tanh', 'sigmoid'])
                )
            )
            model.add(Dropout(hp.Choice('dropout' + str(i), values= [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])))
        counter += 1
        
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(optimizer= hp.Choice('optimizer', values= ['rmsprop', 'adam', 'sgd', 'nadam', 'adadelta']),
                  loss= 'binary_crossentropy',
                  metrics= ['accuracy'])
    
    return model
    
    

In [61]:
tuner = kt.RandomSearch(build_model,
                        objective= 'val_accuracy',
                        max_trials= 3,
                        directory= 'mydir',
                        project_name= 'diabetes1')

c:\Users\disha\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [62]:
tuner.search(X_train, y_train, epochs= 5, validation_data= (X_test, y_test))

Trial 3 Complete [00h 00m 06s]
val_accuracy: 0.3571428656578064

Best val_accuracy So Far: 0.7857142686843872
Total elapsed time: 00h 00m 12s


In [63]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 1,
 'units0': 128,
 'activaation0': 'relu',
 'dropout0': 0.3,
 'optimizer': 'adam'}

In [64]:
model = tuner.get_best_models(num_models= 1)[0]

c:\Users\disha\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\saving\saving_lib.py:713: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [65]:
model.fit(X_train, y_train, epochs= 200, initial_epoch= 5, validation_data= (X_train, y_train))

Epoch 6/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.7498 - loss: 0.4985 - val_accuracy: 0.7638 - val_loss: 0.4761
Epoch 7/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7541 - loss: 0.4737 - val_accuracy: 0.7785 - val_loss: 0.4625
Epoch 8/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7602 - loss: 0.4829 - val_accuracy: 0.7834 - val_loss: 0.4546
Epoch 9/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7905 - loss: 0.4769 - val_accuracy: 0.7785 - val_loss: 0.4492
Epoch 10/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7763 - loss: 0.4455 - val_accuracy: 0.7834 - val_loss: 0.4457
Epoch 11/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7797 - loss: 0.4551 - val_accuracy: 0.7850 - val_loss: 0.4423
Epoch 12/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7789 - loss: 0.4452 - val_accuracy: 0.7850 - val_loss: 0.4389
Epoch 13/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7813 - loss: 0.4533 - val_accuracy: 0.783